# Prepare the base

#### Import data and python libraries

In [300]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [301]:
base = pd.read_excel('/content/sample_data/table.xlsx', sheet_name = 'Planilha1')

#### Transform data

In [302]:
base_test = base

In [303]:
base_test['a'] = (base_test.iloc[:,8] - base_test.iloc[:,9]).dt.days
base_test = base_test[[base_test.iloc[:,18].name, base_test.iloc[:,4].name, base_test.iloc[:,5].name, base_test.iloc[:,19].name, base_test.iloc[:,16].name]]
base_test.iloc[:,4] =  base_test.iloc[:,4].fillna('not canceled')
base_test = pd.get_dummies(base_test, columns = [base_test.iloc[:,1].name],dtype=int)

In [304]:
# Separate the column for one hot
categorical_attributes = [base_test.iloc[:,1].name]
categorical_columns = base_test[categorical_attributes]
# instance and train OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(categorical_columns)
# Codify the categorical columns and transform in a dataframe
encoded = encoder.transform(categorical_columns).toarray()
enc_train = pd.DataFrame(data = encoded, columns = encoder.categories_)
# Concat with the original dataframe and drop original column
base_test = pd.concat([base_test,enc_train],axis=1)
base_test.drop(categorical_attributes, axis=1, inplace=True)

In [305]:
# Separate the column for one hot
categorical_attributes = [base_test.iloc[:,2].name]
categorical_columns = base_test[categorical_attributes]
# instance and train OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(categorical_columns)
# Codify the categorical columns and transform in a dataframe
encoded = encoder.transform(categorical_columns).toarray()
enc_train = pd.DataFrame(data = encoded, columns = encoder.categories_)
# Concat with the original dataframe and drop original column
base_test = pd.concat([base_test,enc_train],axis=1)
base_test.drop(categorical_attributes, axis=1, inplace=True)


In [306]:
# Rename columns
base_test.columns

Index([       'NOME_ENDOSSER',                    'a',           'kind_goods',
              'kind_services',            ('active',),          ('canceled',),
                ('finished',),      ('not canceled',), ('operational_error',),
                  ('others',),          ('reversal',)],
      dtype='object')

In [307]:
base_test = base_test.rename(columns={ base_test.iloc[:,4].name: str(base_test.iloc[:,4].name)[2:-3], base_test.iloc[:,5].name: str(base_test.iloc[:,5].name)[2:-3],base_test.iloc[:,6].name: str(base_test.iloc[:,6].name)[2:-3], base_test.iloc[:,7].name: str(base_test.iloc[:,7].name)[2:-3], base_test.iloc[:,8].name: str(base_test.iloc[:,8].name)[2:-3], base_test.iloc[:,9].name: str(base_test.iloc[:,9].name)[2:-3], base_test.iloc[:,10].name: str(base_test.iloc[:,10].name)[2:-3]})

In [308]:
base_test.isna().sum()

,0
NOME_ENDOSSER,9553
a,0
kind_goods,0
kind_services,0
active,0
canceled,0
finished,0
not canceled,0
operational_error,0
others,0


In [309]:
function_dictionary = {base_test.iloc[:,1].name:'mean',base_test.iloc[:,2].name:'sum',base_test.iloc[:,3].name:'sum', base_test.iloc[:,4].name:'sum', base_test.iloc[:,5].name:'sum', base_test.iloc[:,6].name: 'sum', base_test.iloc[:,7].name: 'sum', base_test.iloc[:,8].name: 'sum',	base_test.iloc[:,9].name: 'sum',	base_test.iloc[:,10].name: 'sum'}
# base_crazy = base_test
base_test = base_test.groupby(base_test.iloc[:,0].name).aggregate(function_dictionary).reset_index(0)

In [310]:
base_nfe = base

In [311]:
base_nfe = base_nfe.groupby([base_nfe.iloc[:,18].name, base_nfe.iloc[:,2].name]).agg({base_nfe.iloc[:,0].name: pd.Series.nunique}).reset_index(0)

In [312]:
base_nfe = base_nfe.reset_index(0)

In [313]:
base_nfe = base_nfe.groupby(base_nfe.iloc[:,1].name).agg({base_nfe.iloc[:,2].name: 'median'}).reset_index(0)

In [314]:
base_test = base_test.set_index(base_nfe.iloc[:,0].name).join(base_nfe.set_index(base_nfe.iloc[:,0].name)).reset_index(0)

In [315]:
base_test = base_test.rename(columns={base_test.iloc[:,11].name: "installment median"})

In [316]:
base_test.fillna(base_test[base_test.iloc[:,11].name].median(), inplace=True)

In [317]:
base_test['t'] = base_test.iloc[:,4] + base_test.iloc[:,5] + base_test.iloc[:,6]

In [318]:
base_test['pnc'] = base_test.iloc[:,7] / base_test.iloc[:,12]

#### Create scoring

In [319]:
base_test['score_f'] = np.where(
    base_test.iloc[:,6] <= 5,(1000*base_test.iloc[:,6])/5,
    np.where(
        (base_test.iloc[:,6] > 5) & (base_test.iloc[:,6] <= 18),(3000*base_test.iloc[:,6])/18,
    np.where(
        (base_test.iloc[:,6] > 18) & (base_test.iloc[:,6] <= 30), (4999*base_test.iloc[:,6])/30,
        5000
    )
        )
    )

In [320]:
base_test['score_a'] = np.where(
    base_test.iloc[:,4] <= 40,(600*base_test.iloc[:,4])/40,
    np.where(
        (base_test.iloc[:,4] > 40) & (base_test.iloc[:,4] <= 199),(1800*base_test.iloc[:,4])/199,
    np.where(
        (base_test.iloc[:,4] > 199) & (base_test.iloc[:,4] <= 484), (2999*base_test.iloc[:,4])/484,
        3000
    )
        )
    )

In [321]:
base_test['score_t'] = np.where(
    base_test.iloc[:,12] <= 40,(400*base_test.iloc[:,12])/40,
    np.where(
        (base_test.iloc[:,12] > 40) & (base_test.iloc[:,12] <= 100),(1200*base_test.iloc[:,12])/100,
    np.where(
        (base_test.iloc[:,12] > 100) & (base_test.iloc[:,12] <= 200), (1999*base_test.iloc[:,12])/200,
        2000
    )
        )
    )

In [322]:
base_test['score_c'] = np.where(
    base_test.iloc[:,5] <= 13,(200*base_test.iloc[:,5])/13,
    np.where(
        (base_test.iloc[:,5] > 13) & (base_test.iloc[:,5] <= 35),(600*base_test.iloc[:,5])/35,
    np.where(
        (base_test.iloc[:,5] > 35) & (base_test.iloc[:,5] <= 65), (999*base_test.iloc[:,5])/65,
        1000
    )
        )
    )

In [323]:
base_test.iloc[:,17]

,score_c
0,0.000000
1,0.000000
2,30.769231
3,0.000000
4,0.000000
...,...
604,0.000000
605,0.000000
606,0.000000
607,0.000000


In [324]:
base_test['s'] = base_test.iloc[:,14] + base_test.iloc[:,15] + base_test.iloc[:,16] - base_test.iloc[:,17]

# IA - MODEL Nº1


In [336]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score # Utilizado para separar dados de treino e teste
from sklearn.preprocessing import StandardScaler # Utilizado para fazer a normalização dos dados
from sklearn.preprocessing import MinMaxScaler # Utilizado para fazer a normalização dos dados
from sklearn.preprocessing import LabelEncoder # Utilizado para fazer o OneHotEncoding
from sklearn.linear_model import LinearRegression # Algoritmo de Regressão Linear
from sklearn.metrics import r2_score, accuracy_score,confusion_matrix, ConfusionMatrixDisplay # Métricas de avaliação do models
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn import tree

In [337]:
# Separate data for use
base_model = base_test[[base_test.iloc[:,1].name,	base_test.iloc[:,2].name,	base_test.iloc[:,3].name,base_test.iloc[:,4].name,	base_test.iloc[:,5].name,	base_test.iloc[:,6].name,	base_test.iloc[:,7].name,	base_test.iloc[:,11].name,	base_test.iloc[:,12].name,base_test.iloc[:,18].name]]
base_model = base_model[base_test.iloc[:,8] <= 1000]


In [338]:
base_model.describe()

,a,kind_goods,kind_services,active,canceled,finished,not canceled,installment median,t,s
count,608.000000,608.000000,608.000000,608.000000,608.000000,608.000000,608.000000,608.000000,608.000000,608.000000
mean,57.821615,47.429276,1.449013,46.241776,1.845395,0.791118,47.032895,1.926809,48.878289,836.270755
std,411.391425,121.659848,14.253785,120.329212,10.442079,3.969503,121.579835,1.833467,122.063525,1340.600936
min,-168.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-250.000000
25%,12.000000,2.000000,0.000000,2.000000,0.000000,0.000000,2.000000,1.000000,3.000000,75.000000
50%,26.000000,10.000000,0.000000,9.000000,0.000000,0.000000,9.000000,1.000000,10.000000,250.000000
75%,42.466667,38.000000,0.000000,36.000000,0.000000,0.000000,36.250000,2.500000,39.250000,931.251256
max,9473.000000,1375.000000,312.000000,1375.000000,125.000000,68.000000,1375.000000,22.500000,1375.000000,9918.448347


In [339]:
X = base_model.loc[ : , base_model.columns != base_model.iloc[:,9].name]
y = base_model.iloc[:,9]

In [340]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [341]:
# Normalization
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [342]:
X_train

array([[0.05445966, 0.00727273, 0.        , ..., 0.00727273, 0.        ,
        0.00655022],
       [0.0591003 , 0.00290909, 0.        , ..., 0.00290909, 0.04651163,
        0.00218341],
       [0.05638731, 0.00581818, 0.        , ..., 0.00581818, 0.02325581,
        0.00509461],
       ...,
       [0.05428164, 0.056     , 0.        , ..., 0.056     , 0.        ,
        0.05531295],
       [0.0587462 , 0.14472727, 0.        , ..., 0.11854545, 0.04651163,
        0.1441048 ],
       [0.06375835, 0.02981818, 0.        , ..., 0.02981818, 0.09302326,
        0.02911208]])

In [343]:
# Treina o modelo
model = LinearRegression()
model = model.fit(X_train, y_train)

In [344]:
# Accuracy
r2_score(y_test, model.fit(X_train, y_train).predict(X_test))

0.7587872477117485

In [345]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Previsões
y_pred = model.predict(X_test)

# Cálculo das métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Exibição dos resultados
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R²: {r2}')

MAE: 430.2382487137543
MSE: 564197.1086312056
RMSE: 751.1305536530954
R²: 0.7587872477117485


In [346]:
# Try to predict score
v1 = 0
v2 = 520
v3 = 19
v4 = 471
v5 = 0
v6 = 68
v7 = 539
v8 = 0
v9 = 539

new_test = [v1,v2,v3,v4,v5,v6,v7,v8,v9]


X = np.array(new_test).reshape(1,-1)
X = sc.transform(X)
print("Score do endossante:", model.predict(X))

Score do endossante: [15783.26784349]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [347]:
pesos = model.coef_
intercepto = model.intercept_
print( pesos)
print("Intercepto:", intercepto)

[  41.19053505 2329.52172654  616.95232748 2321.31436863 -404.47927044
 4689.81556731 2413.40529249  728.99534775 2378.36416912]
Intercepto: 338.7041803434835
